In [1]:
from numpy import fabs
from numpy.core.numeric import outer
from util.data_loader import BCI2aDataset, PhysioDataset
from util.transforms import filterBank, gammaFilter, MSD, Energy_Wavelet, Normal
from torchvision import transforms
from torch.utils.data import DataLoader
from model.svm import SVM
from model.hmm import HMM
from model.energyNN import NeuralNetwork
from model.csp_lda import CSP_LDA
from model.mixed_fbcnet import MIXED_FBCNet
from model.cp_mixednet import CP_MixedNet
from model.mi_cnn import MI_CNN
from model.cnn_lstm import CNNLSTM
#from model.min2net import MIN2NET
from model.eegnet import EEGNet
from model.fbcnet import FBCNet
from torch import nn
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score, det_curve

import os
import torch
import numpy as np
import sys
import yaml

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

CONFIG_PATH = "../config/"


Using cpu device


In [3]:
# Function to load yaml configuration file
def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)

    return config

In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device, dtype=torch.float), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%\n")

def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device, dtype=torch.float), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    epoch_acc = 100.0*correct
    return epoch_acc

def valid(dataloader, model):
    model.eval()
    scores = []
    all_y= []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device, dtype=torch.float), y.to(device)
            pred = model(X)
            scores.extend(10**pred.cpu().numpy()[:,1:])
            all_y.extend(y.cpu().numpy())
    for i, score in enumerate(scores):
        scores[i] = score[0]
    fpr, fnr, thresholds = det_curve(all_y, scores, pos_label=1)
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    print("EER: {}%".format(EER*100))

In [5]:
if __name__ == '__main__':
    config = load_config("mixed_fbcnet_config.yaml")

    if config["transform"]["name"] == "nBand":    
        filterTransform = filterBank([[4,8],[8,12],[12,16],[16,20],[20,24],[24,28],[28,32],[32,36],[36,40]], 160)
    
    
    if config["dataset"]["name"] == "PhysioDataset":
        train_data = PhysioDataset(subject=config["train"]["subject"], path=config["dataset"]["location"], train="train", transform=filterTransform, select_channel=config["channel"]["select"], use_channel_no=config["channel"]["number"], preprocess=config["dataset"]["preprocess"])
        if config["train"]["batch_size"] == "all":
            batch_size = len(train_data)
        else:
            batch_size = config["train"]["batch_size"]
        train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True) 
        
        channels = train_data.channels()

        intra_test_data = PhysioDataset(subject=config["train"]["subject"], path=config["dataset"]["location"], train="intra_test", transform=filterTransform, channels=channels, preprocess=config["dataset"]["preprocess"])
        if config["evaluate"]["batch_size"] == "all":
            batch_size = len(intra_test_data)
        else:
            batch_size = config["evaluate"]["batch_size"]
        intra_test_dataloader = DataLoader(intra_test_data, batch_size=batch_size, shuffle=True, drop_last=True)

        inter_test_data = PhysioDataset(subject=config["train"]["subject"], path=config["dataset"]["location"], train="inter_test", transform=filterTransform, channels=channels, preprocess=config["dataset"]["preprocess"])
        if config["evaluate"]["batch_size"] == "all":
            batch_size = len(inter_test_data)
        else:
            batch_size = config["evaluate"]["batch_size"]
        inter_test_dataloader = DataLoader(inter_test_data, batch_size=batch_size, shuffle=True, drop_last=True)

    
    # save channels
    with open(str(config["train"]["subject"])+'.txt', 'w') as filehandle:
        for channel in channels:
            filehandle.write('%s\n' % channel)

    if config["model"]["name"] in ["FBCNet", "MICNN", "CNN_LSTM", "CP_MIXEDNET", "EEGNet", "MIXED_FBCNet"]:
        if config["model"]["name"] == "MIXED_FBCNet":
            model = MIXED_FBCNet(nChan=config["channel"]["number"]).to(device)
        

        loss_fn = nn.NLLLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config["optimizer"]["initial_lr"], weight_decay=config["optimizer"]["weight_decay"])
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.2)
        best_intra_acc = 0
        best_inter_acc = 0

In [8]:
type(intra_test_data)

util.data_loader.PhysioDataset

In [9]:
model = MIXED_FBCNet(nChan=10).to(device)
model.load_state_dict(torch.load("../trained/4_96.8_96.0_20.pth")) # Load the weights
model.to(device) # Send the model to device
model.eval() # Set the model to evaluation mode

MIXED_FBCNet(
  (channelProj): Sequential(
    (0): Conv2dWithConstraint(10, 30, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (shapeTrans): Sequential(
    (0): Conv2dWithConstraint(30, 30, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (spatialConv): Sequential(
    (0): Conv2dWithConstraint(9, 288, kernel_size=(10, 1), stride=(1, 1), groups=9)
    (1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (pointWise): Sequential(
    (0): Conv2dWithConstraint(288, 288, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (varLayer): LogVarLayer()
  (standTemporalLayer): Sequential(
    (0): Dropout2d(p=0.5, inplace=Fals

In [10]:
# Assuming 'intra_test_data' is your data loader
data_iter = iter(intra_test_data)
single_sample = next(data_iter)

# Unpack the data and labels
data, target = single_sample

# Move the data and label to GPU if available
data = data.to(device)
target = target.to(device)

### Dataset info 

train: run = ["04", "06", "08", "10"]
            
intra / inter  : run = ["12", "14"]

BECAUSE , dataset description (thats why he mentions 6 runs !! apparently each .edf is 15 trials ??? each trial has an imagination segment):

In summary, the experimental runs were:

1   Baseline, eyes open

2   Baseline, eyes closed

3   Task 1 (open and close left or right fist)

4   Task 2 (imagine opening and closing left or right fist) -------------

5   Task 3 (open and close both fists or both feet)

6   Task 4 (imagine opening and closing both fists or both feet) ------------

7   Task 1

8   Task 2 --------------

9   Task 3

10  Task 4 ---------------

11  Task 1

12  Task 2 ---------------
 
13  Task 3

14  Task 4 ----------------



In [19]:
data.shape

torch.Size([10, 640, 9])

dim : torch.Size([10, 640, 9])

From article : 10 channels , 640 time points, 9 

In [23]:
target


1

what is 1 ? 1 is true 0 is false (bona fide vs attack) they use Run 12 and 14 for non train lab=1.

then they add attackers which are not subject 4: for intra we are adding subject 1-21 except 4 as attackers.

hence 20 attackers and 2 bona fide ?

In [17]:
single_sample, label = next(iter(intra_test_data))
single_sample = single_sample.to(device)

# Predict
with torch.no_grad():
    output = model(single_sample.unsqueeze(0))

print(f"Raw output: {output}")

# Convert raw output to predicted class index
predicted_class = output.argmax(dim=1)

print(f"Predicted class: {predicted_class.item()}")

Raw output: tensor([[-3.8718, -0.0210]])
Predicted class: 1


In [6]:
if __name__ == '__main__':
        
        for t in range(config["train"]["epochs"]):
            print(f"Epoch {t+1}\n-------------------------------")
            train(train_dataloader, model, loss_fn, optimizer)
            exp_lr_scheduler.step()
            if (t > 50):
                intra_acc = test(intra_test_dataloader, model, loss_fn, t)
                inter_acc = test(inter_test_dataloader, model, loss_fn, t)
                if t > 250:
                    best_inter_acc = inter_acc
                    valid(intra_test_dataloader, model)
                    valid(inter_test_dataloader, model)
                    #torch.save(model.state_dict(), "../trained/"+str(config["train"]["subject"])+"_"+str(intra_acc)+"_"+str(inter_acc)+"_20"+".pth")
        print("Done!")

Epoch 1
-------------------------------
loss: 0.723920  [    0/  155]


c:\Users\Jacobo\anaconda3\envs\eegBIN\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 0.273518  [   40/  155]
loss: 0.303911  [   80/  155]
loss: 1.015434  [  120/  155]
Train Error: 
 Accuracy: 67.1%

Epoch 2
-------------------------------
loss: 0.153949  [    0/  155]
loss: 0.953507  [   40/  155]
loss: 0.423350  [   80/  155]
loss: 0.556763  [  120/  155]
Train Error: 
 Accuracy: 73.5%

Epoch 3
-------------------------------
loss: 0.268919  [    0/  155]
loss: 0.476662  [   40/  155]
loss: 0.185683  [   80/  155]
loss: 0.303036  [  120/  155]
Train Error: 
 Accuracy: 81.3%

Epoch 4
-------------------------------
loss: 0.634289  [    0/  155]
loss: 0.136143  [   40/  155]
loss: 0.480330  [   80/  155]
loss: 0.661132  [  120/  155]
Train Error: 
 Accuracy: 80.6%

Epoch 5
-------------------------------
loss: 0.717558  [    0/  155]
loss: 0.168494  [   40/  155]
loss: 1.286950  [   80/  155]
loss: 0.230716  [  120/  155]
Train Error: 
 Accuracy: 85.2%

Epoch 6
-------------------------------
loss: 0.862148  [    0/  155]
loss: 0.296063  [   40/  155]
loss: 0.04